In [33]:
import nltk
import numpy as np
import pandas as pd
import string
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import unicodedata

In [34]:
#veriseti data frame'ye aktarılıyor 

df = pd.read_csv("data_cleaned.csv", index_col=[0])

In [35]:
df.head()


category  \
text                                                          
Secimden önce "gerekirse savaş çıkarır fakat RT...  ekonomi   
Temel sorun ekonomi. Ülkenin her tarafını etkil...  ekonomi   
Herkese selamlar, bu yazımı birileri okur ya da...  ekonomi   
NÜFUS Problemi\n\nRTE'nin 3 Çocuk önerilerini h...  ekonomi   
Türkiye niye yanlış yapıyor? Kapıları bütün dün...  ekonomi   

                                                                                          cleanedtext  
text                                                                                                   
Secimden önce "gerekirse savaş çıkarır fakat RT...  SecimdenöncegerekirsesavaşçıkarırfakatRTEveAKP...  
Temel sorun ekonomi. Ülkenin her tarafını etkil...  TemelsorunekonomiÜlkeninhertarafınıetkileyenbi...  
Herkese selamlar, bu yazımı birileri okur ya da...  Herkeseselamlarbuyazımıbirileriokuryadaokumazb...  
NÜFUS Problemi\n\nRTE'nin 3 Çocuk önerilerini h...  NÜFUSProblemiRTEnin3Çocukönerilerinihatırlıyor...  
Türkiye niye yanlış yapıyor? Kapıları bütün dün...  TürkiyeniyeyanlışyapıyorKapılarıbütündünyayaaç...

In [36]:
category = ["ekonomi", "sanat", "teknoloji", "politika"]


In [37]:
def etiketle(row):   
    for key,i in enumerate(category):
        if row["category"] == i:
            return key

In [38]:
df["labels"] = df.apply(lambda row:  etiketle(row), axis=1) 


In [39]:
df.head(-5)


category  \
text                                                           
Secimden önce "gerekirse savaş çıkarır fakat RT...   ekonomi   
Temel sorun ekonomi. Ülkenin her tarafını etkil...   ekonomi   
Herkese selamlar, bu yazımı birileri okur ya da...   ekonomi   
NÜFUS Problemi\n\nRTE'nin 3 Çocuk önerilerini h...   ekonomi   
Türkiye niye yanlış yapıyor? Kapıları bütün dün...   ekonomi   
...                                                      ...   
Hepimiz biliyoruz ki Erdoğan'ın karşısına tuval...  politika   
Akp nin iki yüzlü politika kimliğinin (halka ka...  politika   
Bu yazıyı yazmadan önce birkaç gün beklemek ist...  politika   
Merhaba arkadaşlar İsrail-Filistin çatışması ba...  politika   
Bugün Devlet Bahçeli, uzun süre sonra geri adım...  politika   

                                                                                          cleanedtext  \
text                                                                                                    
Secimden önce "gerekirse savaş çıkarır fakat RT...  SecimdenöncegerekirsesavaşçıkarırfakatRTEveAKP...   
Temel sorun ekonomi. Ülkenin her tarafını etkil...  TemelsorunekonomiÜlkeninhertarafınıetkileyenbi...   
Herkese selamlar, bu yazımı birileri okur ya da...  Herkeseselamlarbuyazımıbirileriokuryadaokumazb...   
NÜFUS Problemi\n\nRTE'nin 3 Çocuk önerilerini h...  NÜFUSProblemiRTEnin3Çocukönerilerinihatırlıyor...   
Türkiye niye yanlış yapıyor? Kapıları bütün dün...  TürkiyeniyeyanlışyapıyorKapılarıbütündünyayaaç...   
...                                                                                               ...   
Hepimiz biliyoruz ki Erdoğan'ın karşısına tuval...  HepimizbiliyoruzkiErdoğanınkarşısınatuvaletter...   
Akp nin iki yüzlü politika kimliğinin (halka ka...  Akpninikiyüzlüpolitikakimliğininhalkakarşıvsya...   
Bu yazıyı yazmadan önce birkaç gün beklemek ist...  Buyazıyıyazmadanöncebirkaçgünbeklemekistedimfa...   
Merhaba arkadaşlar İsrail-Filistin çatışması ba...  MerhabaarkadaşlarİsrailFilistinçatışmasıbaşlad...   
Bugün Devlet Bahçeli, uzun süre sonra geri adım...  BugünDevletBahçeliuzunsüresonrageriadımatıpMer...   

                                                    labels  
text                                                        
Secimden önce "gerekirse savaş çıkarır fakat RT...       0  
Temel sorun ekonomi. Ülkenin her tarafını etkil...       0  
Herkese selamlar, bu yazımı birileri okur ya da...       0  
NÜFUS Problemi\n\nRTE'nin 3 Çocuk önerilerini h...       0  
Türkiye niye yanlış yapıyor? Kapıları bütün dün...       0  
...                                                    ...  
Hepimiz biliyoruz ki Erdoğan'ın karşısına tuval...       3  
Akp nin iki yüzlü politika kimliğinin (halka ka...       3  
Bu yazıyı yazmadan önce birkaç gün beklemek ist...       3  
Merhaba arkadaşlar İsrail-Filistin çatışması ba...       3  
Bugün Devlet Bahçeli, uzun süre sonra geri adım...       3  

[495 rows x 3 columns]

In [40]:
X = df.cleanedtext.to_numpy()  #X arrayine temizlenmiş selftextleri koy  (cleaned)
y = df.labels.to_numpy() #Y arrayine kategorileri koy (labels)

In [41]:
unc_dat=[] #unicode data
for data in X:
    unc_dat.append(unicodedata.normalize('NFKD', str(data)).encode('utf-8', 'ignore'))

X_train, X_test, y_train, y_test = train_test_split(  #x ve y train ve test olarak %80 train - %20 test olacak şekilde parçalanıyor
    unc_dat, y, test_size=0.2, random_state=42)

In [42]:

vectorizer = TfidfVectorizer()              
Xtrain=X_train
Xtest=X_test

X_train = vectorizer.fit_transform(X_train)  
X_test = vectorizer.transform(X_test)

In [43]:

model_NB = MultinomialNB()
model_NB.fit(X_train, y_train)

print("Naive Bayes Model\n")

print("Train Accuracy:", model_NB.score(X_train, y_train))      #Model train accuracy
predictions_train_nb = model_NB.predict(X_train)                 
print("Train F1 Score:", f1_score(y_train, predictions_train_nb, average='micro'))  #Train F1 score

print("\n")

print("Test Accuracy:", model_NB.score(X_test, y_test))         #Model test accuracy
predictions_test_nb = model_NB.predict(X_test)            
print("Test F1 Score:", f1_score(y_test, predictions_test_nb, average='micro'))     #Test F1 score

Naive Bayes Model

Train Accuracy: 0.99
Train F1 Score: 0.99


Test Accuracy: 0.61
Test F1 Score: 0.61


In [44]:
#Bu modelin eğitilmesi 2-3 dk sürebilir.
#SVM modeli train verilerini kullanarak eğitiliyor ve test verileri modele gönderilerek başarısı ölçülüyor

model_SVM=svm.SVC(kernel='linear',random_state=0,probability=True)
model_SVM.fit(X_train, y_train)

print("Support Vector Machine Model\n")

print("Train Accuracy:", model_SVM.score(X_train, y_train))   #Model train accuracy
predictions_train_svm = model_SVM.predict(X_train)                   
print("Train F1 Score:", f1_score(y_train, predictions_train_svm, average='micro'))     #Train F1 score

print("\n")

print("Test Accuracy:", model_SVM.score(X_test, y_test))      #Model test accuracy
predictions_test_svm = model_SVM.predict(X_test)                     
print("Test F1 Score:", f1_score(y_test, predictions_test_svm, average='micro'))        #Test F1 score

Support Vector Machine Model

Train Accuracy: 0.99
Train F1 Score: 0.99


Test Accuracy: 0.77
Test F1 Score: 0.7699999999999999


In [45]:
#Kategorilere göre başarı dağılımlarının gösterilmesi

from sklearn.metrics import classification_report

print("NB\n")

print(classification_report(y_test,predictions_test_nb,target_names=category))

print("\n----------------------------------------------------------------\n")
print("SVM\n")

print(classification_report(y_test,predictions_test_svm,target_names=category))


NB

              precision    recall  f1-score   support

     ekonomi       0.93      0.81      0.86        31
       sanat       0.31      1.00      0.47        15
   teknoloji       0.93      0.58      0.72        24
    politika       0.78      0.23      0.36        30

    accuracy                           0.61       100
   macro avg       0.74      0.66      0.60       100
weighted avg       0.79      0.61      0.62       100


----------------------------------------------------------------

SVM

              precision    recall  f1-score   support

     ekonomi       0.90      0.87      0.89        31
       sanat       1.00      0.67      0.80        15
   teknoloji       0.88      0.58      0.70        24
    politika       0.59      0.87      0.70        30

    accuracy                           0.77       100
   macro avg       0.84      0.75      0.77       100
weighted avg       0.82      0.77      0.77       100



In [46]:
Xtest = [x.decode('UTF8') for x in Xtest] #Xtest arrayinin içindeki tweetler utf-8 formatına dönüştürülüyor


In [47]:
d = {'text': Xtest,'category':y_test ,'nb-tahmin': predictions_test_nb,'svm-tahmin':predictions_test_svm}
df_tahmin = pd.DataFrame(data=d)

In [48]:
df_tahmin.head(-5)


,text,category,nb-tahmin,svm-tahmin
0,SelamherkeseMalumgereksiyasigerekekonomiksebep...,2,2,2
1,ArkadaslarbuhususcokciddiveTRilkdefaodemekrizi...,0,0,0
2,Temellendirilmissiyasibirbilinctuminsantoplulu...,2,1,3
3,Merhabakarantinadadahaönceaklımızdaolanbirşe...,1,1,1
4,ılkdefabubaşlığayazıyorumveeksidesanırımvere...,0,0,0
...,...,...,...,...
90,Ülkedesolcudiyebileceğimizsolekonomikpolitik...,3,1,3
91,CHPLibleftDersimliKemalCnanveonlaraayakuyduran...,3,0,3
92,Merhabageçenpazartesiçindenaldığımteknoloji...,2,2,2
93,Dostlarımsizinleiçtedirginliğimivegeleceğim...,2,1,3


In [49]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


nb_mse=mean_squared_error(y_test,predictions_test_nb) #nb modelinin ortalama kare hatası
print("nb modelinin ortalama kare hatası:",nb_mse)

nb_mae=mean_absolute_error(y_test,predictions_test_nb) #nb modelinin ortalama mutlak hatası
print("nb modelinin ortalama mutlak hatası:",nb_mae)

print("\n")

svm_mse=mean_squared_error(y_test,predictions_test_svm) #svm modelinin ortalama kare hatası
print("svm modelinin ortalama kare hatası:",svm_mse)

svm_mae=mean_absolute_error(y_test,predictions_test_svm) #svm modelinin ortalama mutlak hatası
print("svm modelinin ortalama mutlak hatası:",svm_mae)

nb modelinin ortalama kare hatası: 1.31
nb modelinin ortalama mutlak hatası: 0.67


svm modelinin ortalama kare hatası: 0.91
svm modelinin ortalama mutlak hatası: 0.41


In [50]:
from IPython.display import clear_output 
nltk.download('punkt')     #Lemmatize işlemi için kütüphaneden indiriliyor.

def lemm_selftext(cleanedtext):  
    analyzer = zeyrek.MorphAnalyzer()
    lemm_text = analyzer.lemmatize(cleanedtext)    #selftextler için lemmatize işlemi 
    lemm_text=[el[1][-1] for el in lemm_text]
    clear_text =' '.join(lemm_text)         #dizi string hale getiriliyor
    clear_text = clear_text.lower()         #metin küçük harflere çevriliyor
    clear_output(wait=True)                   #konsol temizleniyor
    return clear_text

[nltk_data] Downloading package punkt to /Users/CANBICH/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [51]:

#text testi

text = "Cumhurbaşkanı Erdoğan Ankaraya açılan operanın açılışına gitti"


lemm_data=lemm_selftext(text) #girilen metni lemmatize işlemine sokarak her kelimesinin kökünü aldırıyoruz

print("TEXT:",text)                          #girilen text
print("LEMM TEXT:",lemm_data,"\n")            #lemmatize edilmiş text

text_vec = vectorizer.transform([lemm_data])  #Metni modele göndermek için metnin vektörünü alıyoruz

category_sonuc_svm=model_SVM.predict(text_vec)   #SVM modeline  texti gönderiyoruz
category_sonuc_nb=model_NB.predict(text_vec)     #MultinomialNB modeline  texti gönderiyoruz

print("NB Model Proba :", model_NB.predict_proba(text_vec))   #Modelin tahmin ettiği olasılıklar
print("SVM Model Proba:", model_SVM.predict_proba(text_vec))    #svm modelinin predict'i proba ya göre daha fazla çaprazlama yaptığı için bazen predict ile proba arasında farklılık oluşabiliyor. 

print("\nNB modelinin tahmini: ", category[category_sonuc_nb[0]])      #NB tahmini
print("SVM modelinin tahmini:", category[category_sonuc_svm[0]])      #SVM  tahmini


TEXT: Cumhurbaşkanı Erdoğan Ankaraya açılan operanın açılışına gitti
LEMM TEXT: cumhurbaşkanı erdoğan ankara açı opera açmak gitmek 

NB Model Proba : [[0.235  0.275  0.2525 0.2375]]
SVM Model Proba: [[0.17378692 0.1272977  0.26718641 0.43172897]]

NB modelinin tahmini:  sanat
SVM modelinin tahmini: politika
